In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras import backend
import matplotlib.pyplot as plt
import pandas
import numpy
import joblib
import math
from datetime import datetime, timedelta

pandas.plotting.register_matplotlib_converters()

In [3]:
def graphicplotxy(x, y, x_label, y_label, title, filename):
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(x, y, alpha=0.70, )
    
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(title)
    ax.legend(['MSE'])
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    ax.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
    
    plt.show
    plt.savefig(filename)

def to_supervised(data, n_in, n_out):
    n_vars = 1
    df = pandas.DataFrame(data)
    cols, names = list(), list()
    # urutan waktu input ke t- (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        
    # urutan waktu output ke t+ (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]          
    # concat
    agg = pandas.concat(cols, axis=1)
    agg.columns = names
    # drop baris yang bernilai Nan
    agg.dropna(inplace=True)
    val = agg.values
    X, y = val[:, 0:n_in], val[:, n_in:]
    #reshape(sample, timestep, feature/input)
    X = X.reshape((X.shape[0], 1, n_in))
    return X, y

def measure_rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return math.sqrt(mse)

def eksperimen(nama, scaler, set_latih, set_uji, n_input, n_neuron, n_epoch):
    n_timestep = 1
    n_output = 1
    
    print("Hyperparameter {0}".format(nama))
    print("input = {0}, neuron = {1}, epoch = {2}".format(n_input, n_neuron, n_epoch))
    
    X_train, y_train = to_supervised(set_latih, n_input, n_output)
    
    model = Sequential() # inisialisasi model
    model.add(LSTM(n_neuron, input_shape=(n_timestep, n_input)))
    model.add(Dense(n_output))
    model.compile(loss='mean_squared_error', optimizer='adam')

    model.fit(X_train, y_train, epochs=n_epoch, verbose=0)
    model.save("{0}.h5".format(nama))

    X_test, y_test = to_supervised(set_uji, n_input, n_output)
    
    saved_model = load_model('{0}.h5'.format(nama))
    y_pred = saved_model.predict(X_test, verbose=0)
    rmse = measure_rmse(y_test, y_pred)
    
    print("{0} RMSE = {1}".format(nama, rmse))
    print("-------------------------------------------------------------------------------------------\n")
    return rmse

In [4]:
#load data
set_latih = numpy.loadtxt('set_latih.csv', delimiter=",")
set_uji = numpy.loadtxt('set_uji.csv', delimiter=",")
scaler = joblib.load('scaler.gz')

In [ ]:
print('Eksperimen MS')

window = [7,30,60,120,365]
neuron = [8,16,32,64,128,256]
epoch = [50,100,200]
ms = pandas.DataFrame(columns=['nama','input','neuron','epoch','rmse', 'time (s)'])
no=1
for i in epoch:
    for j in neuron:
        for k in window:
            nama = "MS{0}".format(no)
            start = datetime.now()
            rmse = eksperimen(nama, scaler, set_latih, set_uji, n_input = k, n_neuron = j, n_epoch = i)
            end = datetime.now()
            time = (end-start).total_seconds()
            ms.loc[no] = [nama,k,j,i,rmse, time]
            ms.to_csv('summary_ms.csv')
            no += 1

Eksperimen MS
Hyperparameter MS1
input = 7, neuron = 8, epoch = 50
MS1 RMSE = 0.25046297654113897
-------------------------------------------------------------------------------------------

Hyperparameter MS2
input = 30, neuron = 8, epoch = 50
MS2 RMSE = 0.2542533647301835
-------------------------------------------------------------------------------------------

Hyperparameter MS3
input = 60, neuron = 8, epoch = 50
MS3 RMSE = 0.2574372966877032
-------------------------------------------------------------------------------------------

Hyperparameter MS4
input = 120, neuron = 8, epoch = 50
MS4 RMSE = 0.2692091914332666
-------------------------------------------------------------------------------------------

Hyperparameter MS5
input = 365, neuron = 8, epoch = 50
MS5 RMSE = 0.30701387013733616
-------------------------------------------------------------------------------------------

Hyperparameter MS6
input = 7, neuron = 16, epoch = 50
MS6 RMSE = 0.2507345344903102
--------------

MS46 RMSE = 0.25046265337537904
-------------------------------------------------------------------------------------------

Hyperparameter MS47
input = 30, neuron = 64, epoch = 100
MS47 RMSE = 0.2850381303206692
-------------------------------------------------------------------------------------------

Hyperparameter MS48
input = 60, neuron = 64, epoch = 100
MS48 RMSE = 0.3030509498933776
-------------------------------------------------------------------------------------------

Hyperparameter MS49
input = 120, neuron = 64, epoch = 100
MS49 RMSE = 0.32142747553388223
-------------------------------------------------------------------------------------------

Hyperparameter MS50
input = 365, neuron = 64, epoch = 100
MS50 RMSE = 0.2779889514566714
-------------------------------------------------------------------------------------------

Hyperparameter MS51
input = 7, neuron = 128, epoch = 100
MS51 RMSE = 0.25066382277624616
-----------------------------------------------------------

In [8]:
print('Eksperimen MS lanjutan, ngecrash Sadge')

window = [7,30,60,120,365]
neuron = [256]
epoch = [200]
ms_lanjutan = pandas.read_csv('summary_ms.csv', index_col=0)
no=86
for i in epoch:
    for j in neuron:
        for k in window:
            nama = "MS{0}".format(no)
            start = datetime.now()
            rmse = eksperimen(nama, scaler, set_latih, set_uji, n_input = k, n_neuron = j, n_epoch = i)
            end = datetime.now()
            time = (end-start).total_seconds()
            ms_lanjutan.loc[no] = [nama,k,j,i,rmse, time]
            ms_lanjutan.to_csv('summary_ms_lanjutan.csv')
            no += 1

Eksperimen MS lanjutan, ngecrash Sadge
Hyperparameter MS86
input = 7, neuron = 256, epoch = 200
MS86 RMSE = 0.2516465535829385
-------------------------------------------------------------------------------------------

Hyperparameter MS87
input = 30, neuron = 256, epoch = 200
MS87 RMSE = 0.34948539128726824
-------------------------------------------------------------------------------------------

Hyperparameter MS88
input = 60, neuron = 256, epoch = 200
MS88 RMSE = 0.35791561209821654
-------------------------------------------------------------------------------------------

Hyperparameter MS89
input = 120, neuron = 256, epoch = 200
MS89 RMSE = 0.3430683525455207
-------------------------------------------------------------------------------------------

Hyperparameter MS90
input = 365, neuron = 256, epoch = 200
MS90 RMSE = 0.27097232209470207
-------------------------------------------------------------------------------------------



In [ ]:
window = [7,30,365]
neuron = [64,128,256]
neuron2 = [8,16,32]
epoch = [50,100,200]
a=0
for i in epoch:
    for j in neuron:
        for k in neuron2:
            for l in window:
                a += 1
print(a)

In [18]:
df = pandas.read_csv('summary_ms_lanjutan.csv', index_col=0)

In [27]:
df.sort_values(by=['rmse'])

,nama,input,neuron,epoch,rmse,time (s)
21,MS21,7,128,50,0.250301,13.126526
16,MS16,7,64,50,0.250408,11.719696
66,MS66,7,16,200,0.250457,42.298422
46,MS46,7,64,100,0.250463,23.998645
1,MS1,7,8,50,0.250463,12.365900
...,...,...,...,...,...,...
87,MS87,30,256,200,0.349485,75.400181
64,MS64,120,8,200,0.349829,49.771052
69,MS69,120,16,200,0.350076,55.681371
88,MS88,60,256,200,0.357916,77.052326


In [2]:
summary = pandas.read_csv('summary_ms_lanjutan.csv', index_col=0)

In [10]:
str(timedelta(seconds=sum(summary['time (s)'].values)))

'1:57:46.882690'

In [11]:
summary.sort_values('rmse')[0:10]

,nama,input,neuron,epoch,rmse,time (s)
21,MS21,7,128,50,0.250301,13.126526
16,MS16,7,64,50,0.250408,11.719696
66,MS66,7,16,200,0.250457,42.298422
46,MS46,7,64,100,0.250463,23.998645
1,MS1,7,8,50,0.250463,12.365900
56,MS56,7,256,100,0.250555,50.330449
26,MS26,7,256,50,0.250635,22.503734
51,MS51,7,128,100,0.250664,27.608562
31,MS31,7,8,100,0.250696,19.583593
6,MS6,7,16,50,0.250735,11.300915


In [ ]:
X_latih = numpy.array([0.1,0.2,0.3]) # set latih X
X_latih = numpy.reshape(X_latih, (1,1,3)) # reshape ke 3 dimensi (jumlah sampel, jumlah timestep, jumlah input/feature)
y_latih = numpy.array([0.4]) # set latih Y
y_latih = numpy.reshape(y_latih, (1,1)) # reshape ke 2 dimensi (jumlah sampel, jumlah output)

model = Sequential() # inisialisasi model
model.add(LSTM(32, input_shape=(X_latih.shape[1], X_latih.shape[2]))) # menambahkan hidden layer LSTM
model.add(Dense(y_latih.shape[1])) # menambahkan output layer
model.compile(loss='mse', optimizer='adam') # compile model dengan loss function mse dan optimizer adam

model.fit(X_latih, y_latih, epochs=100) # training model dengan epoch sebanyak 100
model.save("modelku.h5") # menyimpan model

X_uji = numpy.array([0.2,0.3,0.4]).astype(numpy.float32) # set latih X
X_uji = numpy.reshape(X_uji, (1,1,3)) # reshape ke 3 dimensi (jumlah sampel, jumlah timestep, jumlah input/feature)
y_uji = numpy.array([0.5]).astype(numpy.float32) # set latih Y
y_uji = numpy.reshape(y_latih, (1,1)) # reshape ke 2 dimensi (jumlah sampel, jumlah output)

history = model.evaluate(X_uji, y_uji) # testing model
print('rmse', history) # print rmse